<h2><center>CM3070 Computer Science Final Project</center></h2>
<h2><center>Ng Ka Hou (190387028)</center></h2>

In [1]:
# libraries for linear regression
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
import statsmodels.api as sm
import glob
import os

# libraries for tensorflow
import tensorflow as tf
import keras
from keras import regularizers
from tensorflow.keras import models, layers

# libraries for Grid Search
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from scikeras.wrappers import KerasRegressor

# Helpers
import copy
import math

C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Prepare Data ##

### Load Data ###

In [2]:
# read the dataset of exchange rate
df_exh_rate=pd.read_csv('./data/FRB_H10.csv')

In [3]:
# remove the first 4 rows
df_exh_rate = df_exh_rate.iloc[5:]

In [4]:
df_exh_rate.head()

,Series Description,"UNITED KINGDOM -- SPOT EXCHANGE RATE, US$/POUND (1/RXI_N.B.UK)"
5,1971-01-04,2.3938
6,1971-01-05,2.3949
7,1971-01-06,2.3967
8,1971-01-07,2.3963
9,1971-01-08,2.3972


In [5]:
# change column name
df_exh_rate.columns = ['date', 'gbp_per_usd']

In [6]:
df_exh_rate.reset_index(drop=True, inplace=True)

In [7]:
df_exh_rate.head()

,date,gbp_per_usd
0,1971-01-04,2.3938
1,1971-01-05,2.3949
2,1971-01-06,2.3967
3,1971-01-07,2.3963
4,1971-01-08,2.3972


In [8]:
# read the dataset of usd yield curve
df_usd_int=pd.read_csv('./data/yield-curve-rates-1990-2023.csv')

In [9]:
df_usd_int.head()

,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,12/29/23,5.60,5.59,5.40,5.41,5.26,4.79,4.23,4.01,3.84,3.88,3.88,4.20,4.03
1,12/28/23,5.57,5.55,5.45,5.42,5.28,4.82,4.26,4.02,3.83,3.84,3.84,4.14,3.98
2,12/27/23,5.55,5.53,5.44,5.42,5.26,4.79,4.20,3.97,3.78,3.81,3.79,4.10,3.95
3,12/26/23,5.53,5.52,5.45,5.44,5.28,4.83,4.26,4.05,3.89,3.91,3.89,4.20,4.04
4,12/22/23,5.54,5.52,5.44,5.45,5.31,4.82,4.31,4.04,3.87,3.92,3.90,4.21,4.05


In [10]:
df_usd_int_test=df_usd_int[['Date', '6 Mo']]

In [11]:
df_usd_int_test.head()

,Date,6 Mo
0,12/29/23,5.26
1,12/28/23,5.28
2,12/27/23,5.26
3,12/26/23,5.28
4,12/22/23,5.31


In [12]:
df_usd_int=df_usd_int[['Date', '6 Mo']]

In [13]:
# keep only the date and the 6-Month USD interst rate
df_usd_int.head()

,Date,6 Mo
0,12/29/23,5.26
1,12/28/23,5.28
2,12/27/23,5.26
3,12/26/23,5.28
4,12/22/23,5.31


In [14]:
# rename columns
df_usd_int.rename(columns={'Date': 'date', '6 Mo': '6m_usd_int_rate'}, inplace=True)

In [15]:
df_usd_int.head()

,date,6m_usd_int_rate
0,12/29/23,5.26
1,12/28/23,5.28
2,12/27/23,5.26
3,12/26/23,5.28
4,12/22/23,5.31


In [16]:
file='./data/gbp_yield_data/GLC Nominal daily data_2016 to present.xlsx'

In [17]:
df_gbp_int=pd.read_excel(file, sheet_name='4. nominal spot curve', skiprows=3)

In [18]:
df_gbp_int.head()

,years:,0.5,1,1.5,2,2.5,3,3.5,4,4.5,...,35.5,36,36.5,37,37.5,38,38.5,39,39.5,40
0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-04,NaN,0.343417,0.469762,0.611093,0.750127,0.879897,0.998858,1.107422,1.206596,...,2.551816,2.540093,2.528709,2.517713,2.507147,2.497049,2.487450,2.478377,2.469853,2.461895
3,2016-01-05,NaN,0.326072,0.452016,0.593339,0.732392,0.862314,0.981657,1.090866,1.190934,...,2.547222,2.535446,2.524007,2.512953,2.502328,2.492170,2.482511,2.473379,2.464796,2.456780
4,2016-01-06,NaN,0.293395,0.401197,0.534982,0.670284,0.797753,0.915034,1.022254,1.120310,...,2.479657,2.467708,2.456107,2.444901,2.434137,2.423852,2.414078,2.404845,2.396174,2.388084


In [19]:
df_gbp_int=df_gbp_int.drop(index=0)

In [20]:
df_gbp_int.head()

,years:,0.5,1,1.5,2,2.5,3,3.5,4,4.5,...,35.5,36,36.5,37,37.5,38,38.5,39,39.5,40
1,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-04,NaN,0.343417,0.469762,0.611093,0.750127,0.879897,0.998858,1.107422,1.206596,...,2.551816,2.540093,2.528709,2.517713,2.507147,2.497049,2.487450,2.478377,2.469853,2.461895
3,2016-01-05,NaN,0.326072,0.452016,0.593339,0.732392,0.862314,0.981657,1.090866,1.190934,...,2.547222,2.535446,2.524007,2.512953,2.502328,2.492170,2.482511,2.473379,2.464796,2.456780
4,2016-01-06,NaN,0.293395,0.401197,0.534982,0.670284,0.797753,0.915034,1.022254,1.120310,...,2.479657,2.467708,2.456107,2.444901,2.434137,2.423852,2.414078,2.404845,2.396174,2.388084
5,2016-01-07,NaN,0.306486,0.411970,0.540998,0.672406,0.797378,0.913423,1.020404,1.118953,...,2.519596,2.507792,2.496322,2.485236,2.474578,2.464386,2.454692,2.445526,2.436909,2.428860


In [21]:
df_gbp_int.tail()

,years:,0.5,1,1.5,2,2.5,3,3.5,4,4.5,...,35.5,36,36.5,37,37.5,38,38.5,39,39.5,40
2192,2024-05-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2193,2024-05-28,4.745348,4.746281,4.633256,4.509136,4.401753,4.313099,4.241701,4.185945,4.144182,...,4.625468,4.614194,4.602530,4.590477,4.578035,4.565204,4.551985,4.538377,4.524380,4.509996
2194,2024-05-29,4.772091,4.786440,4.688452,4.578423,4.482607,4.402885,4.338254,4.287633,4.249800,...,4.743537,4.732074,4.720217,4.707968,4.695329,4.682301,4.668883,4.655077,4.640883,4.626303
2195,2024-05-30,4.736628,4.725816,4.629424,4.524510,4.433929,4.358911,4.298358,4.251228,4.216365,...,4.719720,4.708189,4.696267,4.683957,4.671261,4.658180,4.644715,4.630868,4.616638,4.602029
2196,2024-05-31,4.720087,4.695043,4.584558,4.472636,4.380579,4.306682,4.248079,4.202832,4.169367,...,4.639544,4.627790,4.615654,4.603140,4.590250,4.576986,4.563351,4.549346,4.534971,4.520230


In [22]:
folder_path='./data/gbp_yield_data'

In [23]:
file_pattern = os.path.join(folder_path, '*.xlsx')

In [24]:
# array to store the gbp spot yield curve from each excel file
df_gbp_int_arr=[]

In [25]:
for file in glob.glob(file_pattern):
    df_temp=pd.read_excel(file, sheet_name='4. nominal spot curve', skiprows=3)
    df_temp=df_temp.drop(index=0)
    
    df_gbp_int_arr.append(df_temp)

In [26]:
print(df_gbp_int_arr)

[         years:  0.5          1        1.5          2        2.5          3  \
1    1979-01-01  NaN        NaN        NaN        NaN        NaN        NaN   
2    1979-01-02  NaN  11.446775  11.733149  11.959974  12.133409  12.266092   
3    1979-01-03  NaN  11.521413  11.770976  11.987385  12.160734  12.296541   
4    1979-01-04  NaN  11.517242  11.739487  11.941209  12.108479  12.243433   
5    1979-01-05  NaN  11.463236  11.683237  11.885243  12.055542  12.195494   
...         ...  ...        ...        ...        ...        ...        ...   
1562 1984-12-25  NaN        NaN        NaN        NaN        NaN        NaN   
1563 1984-12-26  NaN        NaN        NaN        NaN        NaN        NaN   
1564 1984-12-27  NaN  10.033471  10.333175  10.517918  10.648845  10.750496   
1565 1984-12-28  NaN  10.122834  10.406778  10.581662  10.705856  10.802283   
1566 1984-12-31  NaN  10.134844  10.409208  10.587056  10.714579  10.813168   

            3.5          4        4.5  ...  20.5  

In [27]:
# convert the array of gbp yield curve to dataframe
df_gbp_int = pd.concat(df_gbp_int_arr, ignore_index=True)

In [28]:
df_gbp_int.head()

,years:,0.5,1,1.5,2,2.5,3,3.5,4,4.5,...,35.5,36,36.5,37,37.5,38,38.5,39,39.5,40
0,1979-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1979-01-02,NaN,11.446775,11.733149,11.959974,12.133409,12.266092,12.368615,12.448401,12.510498,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1979-01-03,NaN,11.521413,11.770976,11.987385,12.160734,12.296541,12.402607,12.485395,12.549727,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1979-01-04,NaN,11.517242,11.739487,11.941209,12.108479,12.243433,12.351444,12.437431,12.505331,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1979-01-05,NaN,11.463236,11.683237,11.885243,12.055542,12.195494,12.309677,12.402443,12.477333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df_gbp_int.tail()

,years:,0.5,1,1.5,2,2.5,3,3.5,4,4.5,...,35.5,36,36.5,37,37.5,38,38.5,39,39.5,40
11845,2024-05-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11846,2024-05-28,4.745348,4.746281,4.633256,4.509136,4.401753,4.313099,4.241701,4.185945,4.144182,...,4.625468,4.614194,4.602530,4.590477,4.578035,4.565204,4.551985,4.538377,4.524380,4.509996
11847,2024-05-29,4.772091,4.786440,4.688452,4.578423,4.482607,4.402885,4.338254,4.287633,4.249800,...,4.743537,4.732074,4.720217,4.707968,4.695329,4.682301,4.668883,4.655077,4.640883,4.626303
11848,2024-05-30,4.736628,4.725816,4.629424,4.524510,4.433929,4.358911,4.298358,4.251228,4.216365,...,4.719720,4.708189,4.696267,4.683957,4.671261,4.658180,4.644715,4.630868,4.616638,4.602029
11849,2024-05-31,4.720087,4.695043,4.584558,4.472636,4.380579,4.306682,4.248079,4.202832,4.169367,...,4.639544,4.627790,4.615654,4.603140,4.590250,4.576986,4.563351,4.549346,4.534971,4.520230


In [30]:
df_gbp_int.columns[1]

0.5

In [31]:
df_gbp_int_test=df_gbp_int[['years:',0.5]]

In [32]:
df_gbp_int_test.tail()

,years:,0.5
11845,2024-05-27,NaN
11846,2024-05-28,4.745348
11847,2024-05-29,4.772091
11848,2024-05-30,4.736628
11849,2024-05-31,4.720087


In [33]:
# keep only the date and the 6-Month gbp yield curve data
df_gbp_int=df_gbp_int[['years:',0.5]]

In [34]:
# rename the columns of the dataset of gbp yield curve
df_gbp_int.rename(columns={'years:': 'date', 0.5: '6m_gbp_int_rate'}, inplace=True)

In [35]:
df_gbp_int.head()

,date,6m_gbp_int_rate
0,1979-01-01,NaN
1,1979-01-02,NaN
2,1979-01-03,NaN
3,1979-01-04,NaN
4,1979-01-05,NaN


In [36]:
df_gbp_int.tail()

,date,6m_gbp_int_rate
11845,2024-05-27,NaN
11846,2024-05-28,4.745348
11847,2024-05-29,4.772091
11848,2024-05-30,4.736628
11849,2024-05-31,4.720087


In [37]:
df_exh_rate['date'] = pd.to_datetime(df_exh_rate['date'])
df_usd_int['date'] = pd.to_datetime(df_usd_int['date'], format='%m/%d/%y')
df_gbp_int['date'] = pd.to_datetime(df_gbp_int['date'])

In [38]:
df_exh_rate.head()

,date,gbp_per_usd
0,1971-01-04,2.3938
1,1971-01-05,2.3949
2,1971-01-06,2.3967
3,1971-01-07,2.3963
4,1971-01-08,2.3972


In [39]:
df_usd_int.head()

,date,6m_usd_int_rate
0,2023-12-29,5.26
1,2023-12-28,5.28
2,2023-12-27,5.26
3,2023-12-26,5.28
4,2023-12-22,5.31


In [40]:
df_gbp_int.head()

,date,6m_gbp_int_rate
0,1979-01-01,NaN
1,1979-01-02,NaN
2,1979-01-03,NaN
3,1979-01-04,NaN
4,1979-01-05,NaN


In [41]:
df=df_exh_rate.merge(df_usd_int, on='date', how='left')

In [42]:
df=df.merge(df_gbp_int, on='date', how='left')

In [43]:
df['date_plus_6m']=df['date'] + pd.DateOffset(months=6)

In [44]:
df['date_plus_6m'] = pd.to_datetime(df['date_plus_6m'])

In [47]:
df=df.merge(df_exh_rate, left_on='date_plus_6m', right_on='date', how='left')

In [48]:
df.head()

,date_x,gbp_per_usd_x,6m_usd_int_rate,6m_gbp_int_rate,date_plus_6m,date_y,gbp_per_usd_y
0,1971-01-04,2.3938,NaN,NaN,1971-07-04,NaT,NaN
1,1971-01-05,2.3949,NaN,NaN,1971-07-05,1971-07-05,ND
2,1971-01-06,2.3967,NaN,NaN,1971-07-06,1971-07-06,2.4181
3,1971-01-07,2.3963,NaN,NaN,1971-07-07,1971-07-07,2.4187
4,1971-01-08,2.3972,NaN,NaN,1971-07-08,1971-07-08,2.4185


In [49]:
df.drop(columns=['date_plus_6m', 'date_y'],inplace=True)

In [51]:
df.rename(columns={'date_x': 'date', 'gbp_per_usd_x': 'gbp_per_usd', 'gbp_per_usd_y': 'gbp_per_usd_plus_6m'}, inplace=True)

In [53]:
# convert all non-numeric data to NaN
for col in df.columns:
    if col != 'date':
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [56]:
# drop all rows with any NaN values
df=df.dropna()

In [57]:
df.reset_index(drop=True, inplace=True)

In [58]:
df.head()

,date,gbp_per_usd,6m_usd_int_rate,6m_gbp_int_rate,gbp_per_usd_plus_6m
0,1990-07-11,1.7980,8.15,13.924546,1.9070
1,1990-07-16,1.8040,7.91,13.757289,1.9047
2,1990-07-17,1.8112,7.91,13.700931,1.9228
3,1990-07-18,1.8205,7.88,13.768089,1.9416
4,1990-07-23,1.8205,7.78,13.729768,1.9550


In [59]:
df.drop(columns=['date']).describe(include='all')

,gbp_per_usd,6m_usd_int_rate,6m_gbp_int_rate,gbp_per_usd_plus_6m
count,5125.000000,5125.000000,5125.000000,5125.000000
mean,1.585477,2.795493,3.756305,1.579050
std,0.196986,2.190738,2.848788,0.201964
min,1.075300,0.020000,-0.155476,1.083200
25%,1.461100,0.400000,0.557534,1.454500
50%,1.583600,2.520000,4.405736,1.574200
75%,1.677600,5.040000,5.554162,1.672200
max,2.110400,8.150000,13.924546,2.110400


In [60]:
df[df['6m_gbp_int_rate']<0]

,date,gbp_per_usd,6m_usd_int_rate,6m_gbp_int_rate,gbp_per_usd_plus_6m
4287,2017-02-23,1.2548,0.66,-0.003626,1.2787
4288,2017-02-24,1.2499,0.65,-0.010630,1.2802
4289,2017-02-28,1.2427,0.69,-0.022693,1.2927
4290,2017-03-01,1.2318,0.79,-0.009784,1.2972
4291,2017-03-06,1.2243,0.83,-0.000679,1.3073
...,...,...,...,...,...
4799,2021-04-14,1.3790,0.04,-0.004000,1.3684
4800,2021-04-15,1.3781,0.04,-0.012649,1.3756
4801,2021-04-19,1.3977,0.04,-0.004082,1.3807
4802,2021-04-20,1.3955,0.04,-0.002760,1.3821


In [61]:
df['6m_gbp_int_rate'].min()

-0.15547590836712244

In [62]:
df.tail()

,date,gbp_per_usd,6m_usd_int_rate,6m_gbp_int_rate,gbp_per_usd_plus_6m
5120,2023-11-22,1.2468,5.42,4.977611,1.2739
5121,2023-11-24,1.2601,5.45,4.998473,1.2746
5122,2023-11-28,1.2713,5.42,4.928856,1.2786
5123,2023-11-29,1.2690,5.38,4.927527,1.2714
5124,2023-11-30,1.2640,5.38,4.962727,1.2739


In [63]:
df['gbp_per_usd']=np.log(df['gbp_per_usd'])
df['6m_usd_int_rate']=np.log(1 + df['6m_usd_int_rate']/100)
df['6m_gbp_int_rate']=np.log(1 + df['6m_gbp_int_rate']/100)
df['gbp_per_usd_plus_6m']=np.log(df['gbp_per_usd_plus_6m'])

In [64]:
df['6m_gbp_int_rate'].min()

-0.0015559689757984585

In [65]:
df.dtypes

date                   datetime64[ns]
gbp_per_usd                   float64
6m_usd_int_rate               float64
6m_gbp_int_rate               float64
gbp_per_usd_plus_6m           float64
dtype: object

In [66]:
df.shape

(5125, 5)

In [67]:
df.isna().any()

date                   False
gbp_per_usd            False
6m_usd_int_rate        False
6m_gbp_int_rate        False
gbp_per_usd_plus_6m    False
dtype: bool

In [68]:
df.head()

,date,gbp_per_usd,6m_usd_int_rate,6m_gbp_int_rate,gbp_per_usd_plus_6m
0,1990-07-11,0.586675,0.078349,0.130366,0.645531
1,1990-07-16,0.590006,0.076127,0.128897,0.644325
2,1990-07-17,0.593990,0.076127,0.128401,0.653782
3,1990-07-18,0.599111,0.075849,0.128992,0.663512
4,1990-07-23,0.599111,0.074922,0.128655,0.670390


In [69]:
df.tail()

,date,gbp_per_usd,6m_usd_int_rate,6m_gbp_int_rate,gbp_per_usd_plus_6m
5120,2023-11-22,0.220580,0.052782,0.048577,0.242083
5121,2023-11-24,0.231191,0.053067,0.048776,0.242632
5122,2023-11-28,0.240040,0.052782,0.048112,0.245766
5123,2023-11-29,0.238229,0.052403,0.048100,0.240119
5124,2023-11-30,0.234281,0.052403,0.048435,0.242083


In [84]:
# Import
# Tensorflow
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
import keras
from keras import regularizers
from tensorflow.keras import models, layers
from sklearn.preprocessing import StandardScaler
# Helpers
import numpy as np
import matplotlib.pyplot as plt
import copy
import math

In [88]:
X=df.drop(columns=['date','gbp_per_usd_plus_6m'])
y=df['gbp_per_usd_plus_6m']

In [89]:
X.head()

,gbp_per_usd,6m_usd_int_rate,6m_gbp_int_rate
0,0.586675,0.078349,0.130366
1,0.590006,0.076127,0.128897
2,0.593990,0.076127,0.128401
3,0.599111,0.075849,0.128992
4,0.599111,0.074922,0.128655


In [90]:
y.head()

0    0.645531
1    0.644325
2    0.653782
3    0.663512
4    0.670390
Name: gbp_per_usd_plus_6m, dtype: float64

In [91]:
X=X.to_numpy()
y=y.to_numpy()

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

In [93]:
X_train

array([[0.44096094, 0.01675878, 0.03687802],
       [0.42310854, 0.0008996 , 0.00260603],
       [0.35037514, 0.06071874, 0.05733568],
       ...,
       [0.39924581, 0.05628581, 0.05206561],
       [0.57554538, 0.04497337, 0.04308269],
       [0.62089902, 0.05021772, 0.04885253]])

In [94]:
X_train.shape

(4100, 3)

In [95]:
X_test.shape

(1025, 3)

In [96]:
y_train.shape

(4100,)

In [97]:
y_test.shape

(1025,)

In [99]:
# Function for plotting validation loss trend after each epoch
def plot_val_loss(epoch_num, history, title):
    val_loss = history.history["val_loss"]
    epochs = range(1, epoch_num + 1)
    plt.plot(epochs, val_loss, "b--",
             label="Validation loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

In [100]:
from sklearn.model_selection import KFold

In [101]:
num_folds = 6
batch_size = 64
no_epochs = 50
verbosity = 0

In [102]:
kfold = KFold(n_splits=num_folds, shuffle=True)

In [105]:
no_epochs = 12
acc_per_fold=[]
loss_per_fold=[]
fold_no = 1

# Grid Search - Batch Size and Number of Epochs

In [110]:
# function to create model
def create_model():
    model = models.Sequential()
    model.add(layers.Dense(64, 
              activation='relu',
              input_shape=(3,)))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam',
               loss='mean_squared_error',
               metrics=['mean_absolute_percentage_error'])
    return model

In [111]:
seed = 7
tf.random.set_seed(seed)

In [112]:
# using the KerasRegressor for the Grid Search
model = KerasRegressor(model=create_model, verbose=0)

#### Tune the batch size and the epochs first ####

In [113]:
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]

In [114]:
param_grid = dict(batch_size=batch_size, epochs=epochs)

In [115]:
param_grid

{'batch_size': [10, 20, 40, 60, 80, 100], 'epochs': [10, 50, 100]}

In [116]:
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    cv=3,
                    scoring='neg_mean_squared_error', 
                    error_score='raise')

In [117]:
try:
    grid_result=grid.fit(X_train, y_train)
    print("Best parameters found: ", grid.best_params_)
    print("Best score: ", grid.best_score_)
except ValueError as e:
    print(f"Error during model fitting: {e}")

C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\s

C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\s

C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\s

Best parameters found:  {'batch_size': 10, 'epochs': 100}
Best score:  -0.0032215680196186953


In [118]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.003222 using {'batch_size': 10, 'epochs': 100}
-0.003850 (0.000210) with: {'batch_size': 10, 'epochs': 10}
-0.003515 (0.000105) with: {'batch_size': 10, 'epochs': 50}
-0.003222 (0.000117) with: {'batch_size': 10, 'epochs': 100}
-0.004031 (0.000177) with: {'batch_size': 20, 'epochs': 10}
-0.003587 (0.000226) with: {'batch_size': 20, 'epochs': 50}
-0.003333 (0.000218) with: {'batch_size': 20, 'epochs': 100}
-0.004102 (0.000161) with: {'batch_size': 40, 'epochs': 10}
-0.003805 (0.000144) with: {'batch_size': 40, 'epochs': 50}
-0.003552 (0.000033) with: {'batch_size': 40, 'epochs': 100}
-0.005224 (0.000493) with: {'batch_size': 60, 'epochs': 10}
-0.003991 (0.000130) with: {'batch_size': 60, 'epochs': 50}
-0.003768 (0.000050) with: {'batch_size': 60, 'epochs': 100}
-0.006082 (0.000634) with: {'batch_size': 80, 'epochs': 10}
-0.003921 (0.000223) with: {'batch_size': 80, 'epochs': 50}
-0.003914 (0.000212) with: {'batch_size': 80, 'epochs': 100}
-0.004892 (0.000593) with: {'batch_size

#### the model performs the best when batch size = 10 and epochs = 100. ####

In [119]:
# rebuild the model with the semi-tunned hyperparameter
model = models.Sequential()
model.add(layers.Dense(64, 
          activation='relu',
          input_shape=(3,)))

model.add(layers.Dense(1, activation='linear'))

model.compile(optimizer='adam',
           loss='mean_squared_error',
           metrics=['mean_absolute_percentage_error'])

# fit the model with the whole training dataset
model.fit(X_train, y_train, 
          epochs=100, 
          batch_size=10,
          verbose = 0)

In [120]:
results = model.evaluate(X_test, y_test)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0030 - mean_absolute_percentage_error: 10.5710 


## Grid Search - More hyperparmeters

In [144]:
def create_model(optimizer='adam', activation='relu', neurons=4):
    model = models.Sequential()
    model.add(layers.Dense(neurons, 
              activation=activation,
              input_shape=(3,)))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer=optimizer,loss='mean_squared_error',metrics=['mean_absolute_percentage_error'])
    return model

In [145]:
seed = 7
tf.random.set_seed(seed)
model = KerasRegressor(model=create_model, verbose=0)

#### Tune batch size, epochs and number of neurons ####

In [146]:
# hyperparameters to be tuned
batch_size = [10, 20, 30, 40, 50, 60, 70, 80, 90]
epochs = [10, 20, 30, 50, 70, 90, 100, 120, 150]
optimizer = ['Adam']
activation = ['relu']
neurons = [4, 8, 16, 32, 64, 128]

param_grid = dict(
    batch_size=batch_size, 
    epochs=epochs,
    model__optimizer=optimizer,
    model__activation=activation,
    model__neurons=neurons
)

In [147]:
param_grid

{'batch_size': [16, 32, 64, 128],
 'epochs': [10, 20, 30, 50, 70, 90, 100, 120, 150],
 'model__optimizer': ['Adam'],
 'model__activation': ['relu'],
 'model__neurons': [4, 8, 16, 32, 64, 128]}

In [148]:
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    cv=3,
                    scoring='neg_mean_squared_error', 
                    error_score='raise')

In [149]:
try:
    grid_result=grid.fit(X_train, y_train)
    print("Best parameters found: ", grid.best_params_)
    print("Best score: ", grid.best_score_)
except ValueError as e:
    print(f"Error during model fitting: {e}")

C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\s

C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\s

C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\s

C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\s

C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\s

...

C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\s

C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\s

C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\s

C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ngkah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\s

Best parameters found:  {'batch_size': 16, 'epochs': 150, 'model__activation': 'relu', 'model__neurons': 128, 'model__optimizer': 'Adam'}
Best score:  -0.002987333058604637


In [150]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.002987 using {'batch_size': 16, 'epochs': 150, 'model__activation': 'relu', 'model__neurons': 128, 'model__optimizer': 'Adam'}
-0.013209 (0.004453) with: {'batch_size': 16, 'epochs': 10, 'model__activation': 'relu', 'model__neurons': 4, 'model__optimizer': 'Adam'}
-0.004345 (0.000515) with: {'batch_size': 16, 'epochs': 10, 'model__activation': 'relu', 'model__neurons': 8, 'model__optimizer': 'Adam'}
-0.004033 (0.000094) with: {'batch_size': 16, 'epochs': 10, 'model__activation': 'relu', 'model__neurons': 16, 'model__optimizer': 'Adam'}
-0.004017 (0.000202) with: {'batch_size': 16, 'epochs': 10, 'model__activation': 'relu', 'model__neurons': 32, 'model__optimizer': 'Adam'}
-0.003893 (0.000171) with: {'batch_size': 16, 'epochs': 10, 'model__activation': 'relu', 'model__neurons': 64, 'model__optimizer': 'Adam'}
-0.003894 (0.000238) with: {'batch_size': 16, 'epochs': 10, 'model__activation': 'relu', 'model__neurons': 128, 'model__optimizer': 'Adam'}
-0.004235 (0.000266) with: {'ba

-0.006597 (0.001760) with: {'batch_size': 128, 'epochs': 10, 'model__activation': 'relu', 'model__neurons': 16, 'model__optimizer': 'Adam'}
-0.006075 (0.000692) with: {'batch_size': 128, 'epochs': 10, 'model__activation': 'relu', 'model__neurons': 32, 'model__optimizer': 'Adam'}
-0.005726 (0.000341) with: {'batch_size': 128, 'epochs': 10, 'model__activation': 'relu', 'model__neurons': 64, 'model__optimizer': 'Adam'}
-0.005173 (0.000231) with: {'batch_size': 128, 'epochs': 10, 'model__activation': 'relu', 'model__neurons': 128, 'model__optimizer': 'Adam'}
-0.008570 (0.002154) with: {'batch_size': 128, 'epochs': 20, 'model__activation': 'relu', 'model__neurons': 4, 'model__optimizer': 'Adam'}
-0.009637 (0.001120) with: {'batch_size': 128, 'epochs': 20, 'model__activation': 'relu', 'model__neurons': 8, 'model__optimizer': 'Adam'}
-0.006920 (0.000666) with: {'batch_size': 128, 'epochs': 20, 'model__activation': 'relu', 'model__neurons': 16, 'model__optimizer': 'Adam'}
-0.004195 (0.000254) 

#### The below hyperparameters give the best result ####
- batch_size: 16
- epochs: 150
- model__activation: relu
- model__neurons: 128
- model__optimizer: Adam

In [151]:
# rebuild the model with the semi-tunned hyperparameter
model = models.Sequential()
model.add(layers.Dense(128, 
          activation='relu',
          input_shape=(3,)))
model.add(layers.Dense(1, activation='linear'))

model.compile(optimizer='adam',
           loss='mean_squared_error',
           metrics=['mean_absolute_percentage_error'])

# fit the model with the whole training dataset
model.fit(X_train, y_train, 
          epochs=150, 
          batch_size=16,
          verbose = 0)

In [153]:
results = model.evaluate(X_test, y_test)

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0030 - mean_absolute_percentage_error: 10.7902


#### The result doesn't improve statictially. ####